## Identify CI projects with a pre-period available to analyze
- 12 Months before CI (analysis point).
- Get activity
- Flag rq2 in table Projects
- Record rq2_analysis_init and rq2_analysis_finish

### !!! After that execute scripts on Notebook 11 to fill metrics_period table with this period before CI

In [53]:
repos = getProjects()

for repo in repos:    
    project = repo[0]
    analysisPoint = repo[1]
    ci_service = repo[2]
    
    
    created = getFirstPRDate(project)
    end_period = analysisPoint
    
    if created is None or end_period is None:
        print('Project: {}  -  created is None or end_period is None'.format(project))
        continue
    
    identifyPRActivity(project,created,end_period)

++++++++++++++++ Months: 12  -  PRs: 146  -  Issues: 0  -  Project: ethereum/mist - Init: 2015-07-13 12:13:55-03:00
++++++++++++++++ Months: 12  -  PRs: 94  -  Issues: 0  -  Project: Netflix/security_monkey - Init: 2014-11-01 15:07:02-03:00
++++++++++++++++ Months: 12  -  PRs: 128  -  Issues: 0  -  Project: pilosa/pilosa - Init: 2016-03-03 17:43:56-03:00
++++++++++++++++ Months: 12  -  PRs: 98  -  Issues: 0  -  Project: mgcrea/angular-strap - Init: 2013-02-03 12:41:15-03:00
++++++++++++++++ Months: 12  -  PRs: 188  -  Issues: 0  -  Project: CouchPotato/CouchPotatoServer - Init: 2015-02-02 13:56:33-03:00
++++++++++++++++ Months: 12  -  PRs: 569  -  Issues: 0  -  Project: jquery/jquery-mobile - Init: 2012-02-01 16:15:54-03:00
++++++++++++++++ Months: 12  -  PRs: 267  -  Issues: 0  -  Project: go-lang-plugin-org/go-lang-idea-plugin - Init: 2013-11-09 18:58:45-03:00


In [52]:
def identifyPRActivity(project,init,finish):
    min_months = 12
    
    analysis_init_date = init
    search_date = analysis_finish_date = finish #- relativedelta(months=1) 
    increment=0
    period = total_issues = total_prs = 0
    
    #print('Project: {} - CI: {}'.format(project,finish))
    #while search_date > init:
    while increment < min_months and search_date > init:
        
        prs = selectPRs(project,search_date.month,search_date.year)
        #print('\t\t --Search Date: {} - Prs: {}'.format(search_date,prs[0]))
        #issues = selectIssues(project,search_date.month,search_date.year)
        #total_issues += issues[0]
        
        period += 1
        total_prs += prs[0]
            
        if prs[0]>0:
            analysis_init_tmp = selectFirstPRMonth(project,search_date)
           
        increment +=1   
        search_date = finish - relativedelta(months=increment)        
        if search_date.month == init.month and search_date.year == init.year:
            search_date = init
    

    #data da primeira PR do mês inicial
    analysis_init_date = analysis_init_tmp 
    analysis_finish_date = finish
    
    
    if period >= min_months:
        print('++++++++++++++++ Months: {}  -  PRs: {}  -  Issues: {}  -  Project: {} - Init: {}'.format(period,total_prs,total_issues,project,analysis_init_date))
        setRQ2Classification(project,True,analysis_init_date,analysis_finish_date)

In [2]:
import psycopg2
import os
import requests 
import time
import pytz    
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import pandas as pd
from scipy import stats
import numpy as np
np.seterr(divide='ignore', invalid='ignore')

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

In [3]:
def connectDB():
    f = open('/home/elieziosoares/psql_pwd.txt', "r")
    pwd = f.readline().replace('\n','')
    
    return psycopg2.connect(user = "postgres",
                              password = pwd,
                              host = "127.0.0.1",
                              port = "5432",
                              database = "Causal_CI_Quality_v3")

In [4]:
def getProjects():
    query = """SELECT repo_name, analysis_point,ci_service From projects where rq1 is true and ci is true and DATE_PART('day', analysis_init-analysis_point) < 30;"""
    
    connection = connectDB()
    cursor = connection.cursor()
    cursor.execute(query)
    result = cursor.fetchall()
    connection.close()
    return result

In [5]:
def getFirstPRDate(project):
    connection = connectDB()
    cursor = connection.cursor()
    
    query = """SELECT closed_at from pullrequests where  
        project_name like %s
        order by closed_at limit 1;"""

    cursor.execute(query, [project])
    row = cursor.fetchone()
    
    cursor.close()
    connection.close()
    if row is not None:
        return row[0]
    else:
        return None

In [6]:
def selectPRs(project,month,year):
    connection = connectDB()
    cursor = connection.cursor()
    
    query = """SELECT COUNT(ID) from pullrequests where  
        date_part('month', closed_at) = %s
        AND date_part('year', closed_at) = %s 
        AND project_name like %s"""

    cursor.execute(query, [month,year,project])
    row = cursor.fetchone()
    
    cursor.close()
    connection.close()
    return row

In [7]:
def selectFirstPRMonth(project,date):
    connection = connectDB()
    cursor = connection.cursor()
    
    query = """SELECT closed_at from pullrequests where  
        date_part('month', closed_at) = %s
        AND date_part('year', closed_at) = %s 
        AND project_name like %s
        order by closed_at limit 1;"""
    
    #cursor.execute(query, [date.month,date.year,project])
    
    #query = """SELECT closed_at from pullrequests where  
    #    closed_at >= %s
    #    AND project_name like %s
    #    order by closed_at limit 1;"""

    cursor.execute(query, [date.month,date.year,project])
    row = cursor.fetchone()
    
    cursor.close()
    connection.close()
    if row is not None:
        return row[0]
    else:
        return None

In [37]:
def setRQ2Classification(repo_name,classification,analysis_init_date,analysis_finish_date):
    query = """UPDATE  projects 
        set rq2 = %s,
        rq2_analysis_init = %s,
        rq2_analysis_finish = %s
        WHERE repo_name like %s"""

    connection = connectDB()
    cursor = connection.cursor()
    cursor.execute(query, [classification,analysis_init_date,analysis_finish_date,repo_name])
    connection.commit()
    cursor.close()
    connection.close()

In [45]:
def selectIssues(project,month,year):
    connection = connectDB()
    cursor = connection.cursor()
    
    query = """select COUNT(ID) from issue where  
        date_part('month', created_at) = %s
        AND date_part('year', created_at) = %s 
        AND repo_name like %s"""

    cursor.execute(query, [month,year,project])
    row = cursor.fetchone()
    
    cursor.close()
    connection.close()
    return row